In [15]:
# TEAM UNI:  Sagat Damani ssd2144 Eric Jhonson  efj2106   Aman Shankar as5171
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys

In [16]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [17]:
# Load the data
data_root_path = './cifar10-hw1/'
X_train_orig, y_train_orig = get_train_data(data_root_path) # this may take a few minutes
X_test_orig = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [18]:
#Perform train-validation split (90-10)
X_train, y_train = X_train_orig, y_train_orig
indices = np.random.permutation(X_train.shape[1])
split_point = int(len(indices)* 0.9)
print(split_point)
training_indices, validation_indices = indices[:split_point], indices[split_point:]

X_train, X_validation = X_train[: , training_indices], X_train[:, validation_indices]
y_train, y_validation = y_train[training_indices], y_train[validation_indices]
print(X_train.shape)
print(X_validation.shape)

45000
(3072, 45000)
(3072, 5000)


In [19]:
X_train = np.transpose(X_train).astype("float32")
y_train = y_train.astype("float32")
print(X_train.shape)
print(y_train.shape)

X_validation = np.transpose(X_validation).astype("float32")
y_validation = y_validation.astype("float32")


(45000, 3072)
(45000,)


In [27]:
def cnn_model_fmine(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu, name='conv1')

  kernel = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
  tf.summary.histogram("conv_layer1",kernel)
  conv1 = tf.layers.batch_normalization(conv1)
  drop1 = tf.layers.dropout(inputs=conv1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=drop1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu, name='conv2')

  kernel = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
  tf.summary.histogram("conv_layer2",kernel)
    
  conv2 = tf.layers.batch_normalization(conv2)
  drop2 = tf.layers.dropout(inputs=conv2, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)
  pool2 = tf.layers.max_pooling2d(inputs=drop2, pool_size=[2, 2], strides=2)

  print(conv2.get_shape().as_list())
  print(pool2.get_shape().as_list())

  # Convolutional Layer #3 and Pooling Layer #3
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu, name="conv3")
    
  kernel = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
  tf.summary.histogram("conv_layer3",kernel)

  conv3 = tf.layers.batch_normalization(conv3)
  drop3 = tf.layers.dropout(inputs=conv3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
  pool3 = tf.layers.max_pooling2d(inputs=drop3, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool3_flat = tf.reshape(pool3, [-1, 4 * 4 * 128])
  dense = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu, name="dense")

  kernel = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense/kernel')[0]
  tf.summary.histogram("dense_layer1",kernel)

  drop4 = tf.layers.dropout(inputs=dense, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=drop4, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=tf.nn.softmax(logits, name="softmax_tensor"))

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)# Train the model

In [28]:
# Train the model
print(X_train.shape)
print(y_train.shape)

my_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fmine, model_dir="/tmp/new_best_1")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
my_classifier.train(
    input_fn=train_input_fn,
    steps=40000)

(45000, 3072)
(45000,)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/new_best_1', '_save_summary_steps': 100}
[100, 16, 16, 64]
[100, 8, 8, 64]
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/new_best_1/model.ckpt.
INFO:tensorflow:loss = 2.32993, step = 1
INFO:tensorflow:global_step/sec: 39.219
INFO:tensorflow:loss = 2.09719, step = 101 (2.551 sec)
INFO:tensorflow:global_step/sec: 41.8739
INFO:tensorflow:loss = 1.99161, step = 201 (2.388 sec)
INFO:tensorflow:global_step/sec: 41.7414
INFO:tensorflow:loss = 1.93751, step = 301 (2.396 sec)
INFO:tensorflow:global_step/sec: 41.7589
INFO:tensorflow:loss = 1.64375, step = 401 (2.395 sec)
INFO:tensorflow:global_step/sec: 41.7843
INFO:tensorflow:loss 

In [29]:
# Evaluate the model and print results (training accuracy)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_results = my_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Starting evaluation at 2017-10-31-03:17:08
INFO:tensorflow:Restoring parameters from /tmp/new_best_1/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2017-10-31-03:17:11
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.992044, global_step = 40000, loss = 0.161719
{'loss': 0.16171852, 'global_step': 40000, 'accuracy': 0.99204445}


In [30]:
# Evaluate the model and print results (validation accuracy)
print(X_validation.shape)
print(y_validation.shape)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_validation},
    y=y_validation,
    num_epochs=1,
    shuffle=False)
eval_results = my_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

(5000, 3072)
(5000,)
[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Starting evaluation at 2017-10-31-03:17:18
INFO:tensorflow:Restoring parameters from /tmp/new_best_1/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2017-10-31-03:17:19
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.7844, global_step = 40000, loss = 0.63676
{'loss': 0.63675976, 'global_step': 40000, 'accuracy': 0.78439999}


In [ ]:
# Part 1: 72.74%
# With AdaOptizer, 3 layers, dropout after each layer, 3x3 kernel size: 78.08%

In [31]:
#Run model on test data and output .npy corresponding to last layer
X_test = np.transpose(X_test_orig).astype("float32")
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    num_epochs=1,
    shuffle=False)
predict_results = list(my_classifier.predict(input_fn=eval_input_fn))
predict_results = np.transpose(np.array(predict_results))
print(predict_results.shape)
np.save('save_result_deepnetwork', predict_results)
print(predict_results[:][:10])

[None, 16, 16, 64]
[None, 8, 8, 64]
INFO:tensorflow:Restoring parameters from /tmp/new_best_1/model.ckpt-40000
(10, 10000)
[[  5.62372152e-03   8.76653125e-04   8.04656327e-01 ...,   1.04960964e-04
    4.64902483e-02   3.37883557e-06]
 [  6.26017805e-03   2.07057819e-02   1.14419257e-04 ...,   3.48947026e-07
    2.64663100e-01   2.89091332e-07]
 [  2.39450228e-03   4.54080691e-06   8.43445063e-02 ...,   7.32759163e-02
    3.15578133e-02   3.26221023e-04]
 ..., 
 [  1.13329617e-02   3.74254824e-07   5.49087534e-03 ...,   3.80854867e-03
    8.98400496e-04   9.70146120e-01]
 [  3.26065980e-02   9.77846742e-01   2.13371404e-02 ...,   4.04301827e-05
    1.97346759e-04   3.32854455e-07]
 [  5.82344830e-03   4.09708533e-04   2.61041248e-04 ...,   3.03022716e-05
    2.88082490e-04   8.79599725e-07]]
